In [1]:
!pip install split-folders matplotlib opencv-python spicy
!pip install split-folders[full]

In [2]:
import numpy as np
import cv2 as cv
import os
import splitfolders
import matplotlib.pyplot as plt

import tensorflow.keras as keras
import tensorflow as tf

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img

from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

In [3]:
splitfolders.ratio("dataset2", output="workplace", seed=1337, ratio=(0.7, 0.2, 0.1), group_prefix=None, move=False)

Copying files: 0 files [00:00, ? files/s]


In [31]:
datagen = ImageDataGenerator()

class_names = ['wifi', 'drone', 'noise']

train_generator = datagen.flow_from_directory( 
    directory="/workspace/train/", 
    classes = class_names,
    target_size=(640, 1280),  
    batch_size=32, 
    class_mode="binary", 
)

valid_generator = datagen.flow_from_directory( 
    directory="/workspace/val/", 
    classes = class_names,
    target_size=(640, 1280), 
    batch_size=32, 
    class_mode="binary", 
)

test_generator = datagen.flow_from_directory( 
    directory="/workspace/test/", 
    classes = class_names,
    target_size=(640, 1280), 
    batch_size=32, 
    class_mode="binary", 
)

Found 0 images belonging to 3 classes.
Found 0 images belonging to 3 classes.
Found 0 images belonging to 3 classes.


In [34]:
resnet_50 = ResNet50(include_top=False, weights='imagenet', input_shape=(640,1280,3))
for layer in resnet_50.layers:
    layer.trainable = False

x = resnet_50.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(512, activation='relu')(x) 
x = layers.Dropout(0.5)(x)
x = layers.Dense(256, activation='relu')(x) 
x = layers.Dropout(0.5)(x)
x = layers.Dense(128, activation='relu')(x) 
x = layers.Dropout(0.5)(x)
x = layers.Dense(64, activation='relu')(x) 
x = layers.Dropout(0.5)(x)
predictions = layers.Dense(5, activation='softmax')(x)
model = Model(inputs = resnet_50.input, outputs = predictions)

In [35]:
def trainModel(model, epochs, optimizer):
    batch_size = 32
    model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model.fit(train_generator, validation_data=valid_generator, epochs=epochs, batch_size=batch_size)

In [36]:
model_history = trainModel(model = model, epochs = 10, optimizer = "Adam")

ValueError: Asked to retrieve element 0, but the Sequence has length 0

In [ ]:
loss_train_curve = model_history.history["loss"]
loss_val_curve = model_history.history["val_loss"]
plt.plot(loss_train_curve, label = "Train")
plt.plot(loss_val_curve, label = "Validation")
plt.legend(loc = 'upper right')
plt.title("Loss")
plt.show()

In [ ]:
acc_train_curve = model_history.history["accuracy"]
acc_val_curve = model_history.history["val_accuracy"]
plt.plot(acc_train_curve, label = "Train")
plt.plot(acc_val_curve, label = "Validation")
plt.legend(loc = 'lower right')
plt.title("Accuracy")
plt.show()

In [ ]:
test_loss, test_acc = model.evaluate(test_generator)
print("The test loss is: ", test_loss)
print("The best accuracy is: ", test_acc*100)